# Applied Data Science

## Projekt Gruppe 6 

Gruppenmitglieder: Reto Alexander Nüesch Erismann, Michelle Daniela Wehrli, Rebecca Sarah Baumann


## Aufgabe 1.1

Datenbezug über API-Schnittstelle von Kaggle, um das Modell zu trainieren, vaidieren und um es zu testen


In [ ]:
# !pip install kaggle

In [ ]:
# !pip install opendatasets

#### Kommentar: Wir haben manuel den Kaggle API-Token aus dem User-Account ins Working Directory des Jupyter-Notebooks heruntergeladen.

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi ()
api.authenticate ()

import opendatasets as od
od.download("https://www.kaggle.com/datasets/sshikamaru/fruit-recognition")

# od.download("https://www.kaggle.com/datasets/sshikamaru/fruit-recognition", force=True) ... falls die Bilder noch einmal heruntergeladen werden sollten

## Aufgabe 1.2

Bilder zur Überprüfung des Models über Web Scraping beziehen.
Inspirierende Quelle: https://github.com/Emmanuel96/scrape-helmets-images-from-google/blob/main/scrape_helmet_images.py

In [2]:
import os
import json 
import requests
import selenium
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
import base64
import time
import urllib.request

DRIVER_PATH = 'C:/Users/User/OneDrive - ZHAW/GitHub/AppliedDataScience/chromedriver.exe'
SAVE_FOLDER = 'C:/Users/User/OneDrive - ZHAW/GitHub/AppliedDataScience/fruit-google/'
GOOGLE_IMAGES = 'https://www.google.com/search?q=fruits+single&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjb2_SOjJb3AhWbif0HHbVpCAoQ_AUoAXoECAEQAw&biw=1920&bih=927&dpr=1'

driver = webdriver.Chrome(DRIVER_PATH)
driver.get(GOOGLE_IMAGES)

# Scroll to the end of the page
def scroll_to_end():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    print('scroll done')

counter = 0
for i in range(1,2):     
    scroll_to_end()
    image_elements = driver.find_elements_by_class_name('rg_i')
    print(len(image_elements))
    for image in image_elements: 
        if (image.get_attribute('src') is not None):
            my_image = image.get_attribute('src').split('data:image/jpeg;base64,')
            filename = SAVE_FOLDER + 'fruit'+str(counter)+'.jpeg'
            if(len(my_image) >1): 
                with open(filename, 'wb') as f: 
                    f.write(base64.b64decode(my_image[1]))
            else: 
                print(image.get_attribute('src'))
                urllib.request.urlretrieve(image.get_attribute('src'), SAVE_FOLDER + 'fruit'+ str(counter)+'.jpeg')
            counter += 1

C:\Users\User\AppData\Local\Temp/ipykernel_23716/3040065594.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


scroll done
100


C:\Users\User\AppData\Local\Temp/ipykernel_23716/3040065594.py:27: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  image_elements = driver.find_elements_by_class_name('rg_i')


https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTIbW2SyS9qdpwyOdntZ6yS-Qi-K1JVbMSDlQ&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQYrY0AogFmcOt_SfHLaQeiK-WApkadosyHUA&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREw4s_05KWcHfuM_YKbt8rJkii0VO-2hHgjA&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_n11axGtCaCKrBWOfpqdX-WySrxoG8y4Pig&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwrhZ6iIwuYgJiJs9Q0ePd6rpZXKRTlYNDIA&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRE3uADXNzNaVhKSztfMCSwqOjjWFJgvVyW4A&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTgcwESw87QLkM2CfxaNDEpcl0Os5veQ19Bkw&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTN4PbdTNEGFn2wN0ow-OjUgyyosN9x-Grh7w&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRFkVsfNaAigaag8zM2cCdrqN4dqrzGHuln8A&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTgFHKfaISUWVLcI8HEnGQJ4mhKMOMfAT6ebw&usqp=CAU
